In [28]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import unidecode
from collections import Counter

In [29]:
from data_cleaning import preparation_data

In [30]:
path = 'Data/' ## Path Solene

df_subventions = preparation_data(path)

In [31]:
df_subventions = pd.read_csv(os.path.join(path, 'bp-2019-ville-de-rennes-subventions-aux-associations.csv'), sep=",")

In [32]:
df_subventions.columns

Index(['Libellé Fonction', 'Sous-fonction', 'Libellé sous-fonction', 'Article',
       'Bénéficiaire', 'Budget de l'année', 'Libellé article'],
      dtype='object')

In [33]:
df_subventions.iloc[0]

Libellé Fonction                                        Sports et jeunesse
Sous-fonction                                                        422.1
Libellé sous-fonction                               Action socio-éducative
Article                                                               6574
Bénéficiaire             AMSIC  (Association Maison des Squares Solidar...
Budget de l'année                                                   202587
Libellé article          Subv. fonct. aux associat.et autres personnes ...
Name: 0, dtype: object

In [34]:
df_subventions = df_subventions.rename({
    "Budget de l'année":'montant', 
#     'Libellé Fonction':'activite_association'
    'Libellé sous-fonction':'activite_association'
}, axis = 'columns')


In [35]:
variables_base = ['montant','activite_association']

In [36]:
df_subventions=df_subventions[variables_base]

In [37]:
# drop rows with no activite_association
float_activities_to_drop = np.where([type(x) == float for x in df_subventions['activite_association']])
df_subventions = df_subventions.drop(float_activities_to_drop[0], axis=0)

In [38]:
df_subventions

,montant,activite_association
0,202587,Action socio-éducative
1,35000,Tranquillité publique prévention de la délinq...
2,21560,Autres crèches
3,26000,Autres actions artistiques et culturelles
4,27093,Autres interventions sociales
5,31800,Actions de prévention sanitaire
6,23000,Handicap et accessibilité
7,30000,Activités éducatives périscolaires
8,8160,Consommation - diverses actions
9,10197,Action socio-éducative


In [39]:
#find all possible activities
df_activities = df_subventions['activite_association'].str.split(pat="/", expand=True)
                                                                 
list_activities = []
list_activities += list(df_activities.loc[:,0].unique())
                                                                 
# unique values
list_activities = list(set(list_activities))
# list_activities.remove(None)

In [40]:
list_activities

['Activités éducatives périscolaires',
 'Autres actions artistiques et culturelles',
 '824.4',
 'Information Communication',
 'Autres crèches',
 'Animation sportive',
 'Bibliothèques',
 'Politique de la Ville',
 'Maitrise énergie',
 'Centres sociaux',
 'Tranquillité publique  prévention de la délinquance',
 'Secteur culturel - manif.coorganisées',
 'Enseignement public 1er degré - maternel',
 'Expression musicale',
 'Relations Publiques',
 "Actions pour l'emploi",
 'Activités théâtrales',
 'Salles culturelles en régie',
 'Commerce - diverses actions',
 'Oeuvres  scolaires',
 'Actions de protection civile',
 'Egalité des droits et diversité',
 'Handicap et accessibilité',
 'Autres interventions  sociales',
 'Espaces verts et jardins',
 '824.1',
 'Actions de prévention sanitaire',
 'Aides au logement social',
 '824.3',
 'Consommation - diverses actions',
 'Aides aux personnes âgées',
 "Centre communal d'action sociale",
 "Actions pour l'environnement",
 'CRR - Musiques',
 "Autres actions

In [41]:
dict_remplacement_categories = {
'Aménagement urbain - opérations directes' : 'Architecture & urbanisme',
 'Actions quartiers': 'Architecture & urbanisme',
 '824.1':'Architecture & urbanisme',
 'Autres charges de personnel':'autre',
 'Tranquillité publique  prévention de la délinquance':'Vie et animation locale',
 "Actions pour l'emploi":'Emploi',
 'Actions de prévention sanitaire':'Santé',
 'Aides aux personnes âgées':'Social',
 'Autres interventions  sociales':'Social',
 'Animation sportive':'Sport',
 'Activités éducatives périscolaires':"Education & formation",
 'CRR - Musiques':"Culture & Arts",
 'Jumelages et Relations Internationales':"Relations internationales",
 'Handicap et accessibilité':"Santé",
 'Aides au logement social':"Précarité & Exclusion",
 'Bibliothèques':"Education & formation",
 'Secteur culturel - manif.coorganisées':"Culture & Arts",
 'Enseignement public 1er  degré - élémentaire':"Education & formation",
 'Egalité des droits et diversité':"Défense des droits et des intérêts",
 'Salles culturelles en régie':"Culture & Arts",
 'Centres sociaux':"Social",
 'Actions de protection civile':"Santé",
 "Actions pour l'environnement":"Environnement & écologie",
 "Centre communal d'action sociale":"Social",
 'Autres actions en faveur des jeunes':"Education & formation",
 '824.3':"Architecture & urbanisme",
 "Autres actions en faveur de l'enfance":"Education & formation",
 'Tranquillité publique':"Aides aux associations",
 'Consommation - diverses actions':"Santé",
 'Circulation voirie': "Architecture & urbanisme",
 'ACCES AUX LOISIRS POUR TOUS : DISPOSITIF SORTIR':"Loisirs",
 'Classes de découvertes':"Education & formation",
 'Espaces verts et jardins':"Environnement & écologie",
 'Enseignement public 1er degré - maternel':"Architecture & urbanisme",
 'Autres crèches':'Social',
 'Secteur sportif - manif.coorganisées':'Sport',
 'Théâtre National de Bretagne':'Culture & Arts',
 'Autres enseignements':'Education & formation',
 'Activités théâtrales':'Culture & Arts',
 'Autres actions artistiques et culturelles':'Culture & Arts',
 'Oeuvres  scolaires':"Education & formation",
 'Actions en faveur des personnes en difficulté':'Précarité & Exclusion',
 'Commerce - diverses actions':'Economie',
 'Action socio-éducative':"Education & formation",
 'Information Communication':"Vie et animation locale",
 'Expression musicale':'Culture & Arts',
 'Maitrise énergie':'Environnement & écologie', 
 'Politique de la Ville':'Architecture & urbanisme',
 '824.4':'Architecture & urbanisme',
 'Relations Publiques':'Communication & média'

}

In [42]:
for cat_montpellier, cat_paris in dict_remplacement_categories.items():
    df_subventions["activite_association"] = df_subventions["activite_association"].str.replace(cat_montpellier, cat_paris)

In [43]:
df_subventions

,montant,activite_association
0,202587,Education & formation
1,35000,Vie et animation locale
2,21560,Social
3,26000,Culture & Arts
4,27093,Social
5,31800,Santé
6,23000,Santé
7,30000,Education & formation
8,8160,Santé
9,10197,Education & formation


In [44]:
list_activites_paris = ['Technique & Recherche',
'Architecture & urbanisme', 
'Social',
'Education & formation',
'Culture & Arts',
'Humanitaire',
'Communication & média',
'Environnement & écologie', 
'Aides aux associations',
'Sport',
'Santé',
'Relations internationales',
'Idée & opinion', 
'Loisirs',
'Déplacements et transports',
'Tourisme',
'Mémoire',
'Défense des droits et des intérêts',
'Economie',
'Emploi',
'Précarité & Exclusion',
'Vie et animation locale']

In [45]:
def get_dummy_activite(activite):
    act = [int(activite in x) for x in list(df_subventions['activite_association'].str.split(pat="/"))]
    return act

In [46]:
# add dummies for all possible activities\n",
for i in list_activites_paris:
    df_subventions[i] = get_dummy_activite(i)

In [47]:
df_subventions

,montant,activite_association,Technique & Recherche,Architecture & urbanisme,Social,Education & formation,Culture & Arts,Humanitaire,Communication & média,Environnement & écologie,...,Idée & opinion,Loisirs,Déplacements et transports,Tourisme,Mémoire,Défense des droits et des intérêts,Economie,Emploi,Précarité & Exclusion,Vie et animation locale
0,202587,Education & formation,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,35000,Vie et animation locale,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,21560,Social,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,26000,Culture & Arts,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,27093,Social,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,31800,Santé,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,23000,Santé,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,30000,Education & formation,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8160,Santé,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10197,Education & formation,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Tidy

In [48]:
# df_subventions['montant'] = df_subventions['montant'].str.replace(r"[a-zA-Z|,| |€]",'').astype(int)

In [49]:
df_subventions=df_subventions[df_subventions['montant']>0]

In [50]:
df_subventions.to_csv('Data/df_rennes_clean.csv', index=False)

# analyse des activités

In [51]:
import plotly.express as px
import plotly
import plotly.graph_objects as go

In [52]:
df_subventions

,montant,activite_association,Technique & Recherche,Architecture & urbanisme,Social,Education & formation,Culture & Arts,Humanitaire,Communication & média,Environnement & écologie,...,Idée & opinion,Loisirs,Déplacements et transports,Tourisme,Mémoire,Défense des droits et des intérêts,Economie,Emploi,Précarité & Exclusion,Vie et animation locale
0,202587,Education & formation,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,35000,Vie et animation locale,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,21560,Social,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,26000,Culture & Arts,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,27093,Social,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,31800,Santé,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,23000,Santé,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,30000,Education & formation,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8160,Santé,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10197,Education & formation,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
var_cols = ['Technique & Recherche',
       'Architecture & urbanisme', 'Social', 'Education & formation',
       'Culture & Arts', 'Humanitaire', 'Communication & média',
       'Environnement & écologie', 'Aides aux associations', 'Sport', 'Santé',
       'Relations internationales', 'Idée & opinion', 'Loisirs',
       'Déplacements et transports', 'Tourisme', 'Mémoire',
       'Défense des droits et des intérêts', 'Economie', 'Emploi',
       'Précarité & Exclusion', 'Vie et animation locale']

df_subventions[var_cols] = df_subventions[var_cols].div(df_subventions[var_cols].sum(axis=1), axis=0)

df_subventions[var_cols] = df_subventions[var_cols].multiply(df_subventions["montant"], axis="index")

final = pd.DataFrame(df_subventions[var_cols].sum())

(100. * final / final.sum()).round(0)

final=final.reset_index()

In [55]:
fig = px.bar(final, x="index",y=0)
fig.update_xaxes(categoryorder = 'total descending')
fig.show()